In [1]:
import os
import glob
import random
from PIL import Image
import numpy as np
import cv2
from tqdm import tqdm
from facenet_pytorch import MTCNN
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator
import torch

In [2]:
import concurrent.futures

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mtcnn = MTCNN(keep_all=False, device=device)
from segment_anything import sam_model_registry
sam = sam_model_registry["vit_b"](checkpoint="sam_vit_b_01ec64.pth").to(device)
mask_generator = SamAutomaticMaskGenerator(
    sam,
    points_per_side=16,  # less sampling = faster
    pred_iou_thresh=0.9,
    stability_score_thresh=0.95,
    crop_n_layers=0,
    min_mask_region_area=500  # ignore tiny masks
)

In [5]:
from multiprocessing import Pool, cpu_count

In [ ]:
CLASS_MAP = {
    'wrinkles': 0,
    'redness': 1,
    'dryness': 2,
    'acne_Level_0': 3,
    'acne_Level_1': 4,
    'acne_Level_2': 5,
}

# Heuristics
def is_wrinkle(region, mask):
    gray = cv2.cvtColor(region, cv2.COLOR_RGB2GRAY)
    masked = cv2.bitwise_and(gray, gray, mask=mask.astype(np.uint8))
    edges = cv2.Canny(masked, 100, 200)
    return np.sum(edges > 0) / mask.sum() > 0.1

def is_redness(region, mask):
    r, g, b = region[:, :, 0], region[:, :, 1], region[:, :, 2]
    redness = (r > 120) & ((r - g) > 20) & ((r - b) > 20)
    return np.sum(redness & mask) > 200

def is_dryness(region, mask):
    gray = cv2.cvtColor(region, cv2.COLOR_RGB2GRAY)
    masked = cv2.bitwise_and(gray, gray, mask=mask.astype(np.uint8))
    return np.std(masked[mask]) < 18

def get_acne_label(image_path):
    for level in ["acne_Level_0", "acne_Level_1", "acne_Level_2"]:
        if level in image_path:
            return CLASS_MAP[level]
    return None

def mask_to_yolo(mask, label_id, img_w, img_h):
    ys, xs = np.where(mask)
    if len(xs) == 0 or len(ys) == 0:
        return None
    x_min, x_max, y_min, y_max = xs.min(), xs.max(), ys.min(), ys.max()
    x_center, y_center = ((x_min + x_max) / 2) / img_w, ((y_min + y_max) / 2) / img_h
    width, height = (x_max - x_min) / img_w, (y_max - y_min) / img_h
    return f"{label_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"

def process_image(image_path):
    try:
        img = Image.open(image_path).convert('RGB')
        img = img.resize((512, 512))
        np_img = np.array(img)

        masks = mask_generator.generate(np_img)
        h, w = np_img.shape[:2]
        yolo_annotations = []

        for m in masks[:10]:
            mask = m['segmentation']
            region = cv2.bitwise_and(np_img, np_img, mask=mask.astype(np.uint8))

            if is_wrinkle(np_img, mask):
                yolo = mask_to_yolo(mask, CLASS_MAP['wrinkles'], w, h)
                if yolo: yolo_annotations.append(yolo)

            if is_redness(np_img, mask):
                yolo = mask_to_yolo(mask, CLASS_MAP['redness'], w, h)
                if yolo: yolo_annotations.append(yolo)

            if is_dryness(np_img, mask):
                yolo = mask_to_yolo(mask, CLASS_MAP['dryness'], w, h)
                if yolo: yolo_annotations.append(yolo)

        acne_label = get_acne_label(image_path)
        if acne_label is not None:
            yolo_annotations.append(f"{acne_label} 0.5 0.5 1.0 1.0")

        return np_img, yolo_annotations
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None, None

def prepare_dataset(image_root='faces_dataset', output_root='yolo_dataset', split_ratio=0.8):
    all_images = glob.glob(f"{image_root}/**/*.jpg", recursive=True)
    random.shuffle(all_images)
    split_idx = int(len(all_images) * split_ratio)
    train_imgs, val_imgs = all_images[:split_idx], all_images[split_idx:]

    for mode, images in zip(['train', 'val'], [train_imgs, val_imgs]):
        img_dir = os.path.join(output_root, 'images', mode)
        lbl_dir = os.path.join(output_root, 'labels', mode)
        os.makedirs(img_dir, exist_ok=True)
        os.makedirs(lbl_dir, exist_ok=True)

        def process_and_save(img_path):
            base = os.path.splitext(os.path.basename(img_path))[0]
            out_img_path = os.path.join(img_dir, base + ".jpg")
            out_lbl_path = os.path.join(lbl_dir, base + ".txt")
            if os.path.exists(out_lbl_path):
                return
            try:
                cropped_img, annotations = process_image(img_path)
                if cropped_img is None or not annotations:
                    return
                Image.fromarray(cropped_img).save(out_img_path)
                with open(out_lbl_path, 'w') as f:
                    f.write("\n".join(annotations))
            except Exception as e:
                print(f"Error in process_and_save({img_path}): {e}")

        with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
            list(tqdm(executor.map(process_and_save, images), total=len(images), desc=f"Processing {mode} set"))
            
if __name__ == '__main__':
   # Step 2: Label the face images
    prepare_dataset(image_root='faces_dataset')

Processing train set:   4%|██                                                     | 39/1054 [35:21<20:38:28, 73.21s/it]

Error processing faces_dataset\acne_Level_2\levle2_97.jpg: An image must be set with .set_image(...) before mask prediction.


Processing train set:  17%|████████▊                                            | 175/1054 [2:41:38<6:31:23, 26.72s/it]

Error processing faces_dataset\acne_Level_0\levle0_371.jpg: An image must be set with .set_image(...) before mask prediction.


Processing train set:  20%|██████████▏                                         | 207/1054 [3:13:45<13:03:22, 55.49s/it]

...

Processing train set:  65%|█████████████████████████████████▌                  | 681/1054 [14:33:19<4:19:14, 41.70s/it]

Error processing faces_dataset\acne_Level_2\levle2_182.jpg: An image must be set with .set_image(...) before mask prediction.


Processing train set:  70%|████████████████████████████████████▎               | 735/1054 [15:31:31<7:34:33, 85.50s/it]

Error processing faces_dataset\wrinkles\000047.jpg: An image must be set with .set_image(...) before mask prediction.
Error processing faces_dataset\acne_Level_0\levle0_227.jpg: An image must be set with .set_image(...) before mask prediction.


Processing train set:  70%|████████████████████████████████████▎               | 737/1054 [15:31:48<5:31:04, 62.66s/it]

In [1]:
!pip install ultralytics

  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
   ---------------------------------------- 0.0/984.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/984.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/984.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/984.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/984.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/984.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/984.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/984.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/984.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/984.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/984.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/984.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/984.0 kB ? eta -:--:--
   ----

In [5]:
!yolo task=detect mode=train model=yolov8n.pt data=dataset.yaml epochs=50 imgsz=640

Ultralytics 8.3.116  Python-3.10.8 torch-2.2.2+cpu CPU (Intel Core(TM) i5-6300U 2.40GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, fo

Traceback (most recent call last):
  File "C:\Users\hamad\AppData\Local\Programs\Python\Python310\lib\site-packages\ultralytics\engine\trainer.py", line 581, in get_dataset
    data = check_det_dataset(self.args.data)
  File "C:\Users\hamad\AppData\Local\Programs\Python\Python310\lib\site-packages\ultralytics\data\utils.py", line 393, in check_det_dataset
    file = check_file(dataset)
  File "C:\Users\hamad\AppData\Local\Programs\Python\Python310\lib\site-packages\ultralytics\utils\checks.py", line 541, in check_file
    raise FileNotFoundError(f"'{file}' does not exist")
FileNotFoundError: 'dataset.yaml' does not exist

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\hamad\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\hamad\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exe